In [65]:
import sys

sys.path.append('/Users/kallewesterling/Dropbox (Graduate Center)/dev-dhri/django-app/')

In [66]:
from pathlib import Path
import json

data = json.loads(Path('../../lessons.json').read_text())

In [116]:
from pathlib import Path
from collections import OrderedDict

In [220]:
class Lesson():
    
    def __len__(self):
        return(len(self.data))
        
    def __init__(self, markdown:str):
        self.markdown = markdown
        
        self.data = []

        for title, body in split_into_sections(self.markdown, level_granularity=1).items():
            droplines = []

            challenge = ""
            # test for challenge
            if "challenge" in body.lower():
                for line_num, line in enumerate(body.splitlines()):
                    if line.lower().startswith("## challenge"):
                        droplines.append(line_num)
                        startline = line_num + 1
                        nextlines = (item for item in body.splitlines()[startline:])
                        done = False
                        while not done:
                            try:
                                l = next(nextlines)
                                if l.startswith("#"):
                                    done = True
                                    continue
                                challenge += l + "\n"
                                droplines.append(startline)
                                startline += 1
                            except StopIteration:
                                done = True

            solution = ""
            # test for solution
            if "solution" in body.lower():
                for line_num, line in enumerate(body.splitlines()):
                    if line.lower().startswith("## solution"):
                        droplines.append(line_num)
                        startline = line_num + 1
                        nextlines = (item for item in body.splitlines()[startline:])
                        done = False
                        while not done:
                            try:
                                l = next(nextlines)
                                if l.startswith("#"):
                                    done = True
                                    continue
                                solution += l + "\n"
                                droplines.append(startline)
                                startline += 1
                            except StopIteration:
                                done = True

            # fix body
            cleaned_body = ''
            for i, line in enumerate(body.splitlines()):
                if line.strip() == '': continue
                if i not in droplines:
                    cleaned_body += line + '\n'
            
            # clean up everything
            title = title.replace("\n", " ").strip()
            cleaned_body = cleaned_body.replace("\n", " ").strip()
            challenge = challenge.replace("\n", " ").strip()
            solution = solution.replace("\n", " ").strip()
            
            if challenge == '': challenge = None
            if solution == '': solution = None
            
            if solution and not challenge:
                raise RuntimeError("What is happening - we have only a solution in this lesson")
            elif challenge and not solution:
                print('we have a challenge with no solution! Raise warning maybe...')
            
            data = {
                    'title': lesson,
                    'body': cleaned_body,
                    'challenge': challenge,
                    'solution': solution
                }
            
            self.data.append(data)

raw_lesson_markdown = Path('../../lessons.txt').read_text()

l = Lesson(raw_lesson_markdown)

we have a challenge with no solution! Raise warning maybe...
we have a challenge with no solution! Raise warning maybe...
we have a challenge with no solution! Raise warning maybe...
we have a challenge with no solution! Raise warning maybe...


In [222]:
from pprint import pprint

for lesson in l.data:
    if lesson['challenge']:
        print(lesson['challenge'])
        print(lesson['solution'])

18

In [252]:
import re
import requests
from dhri.utils.regex import all_images

In [253]:
markdown = Path('../../lessons.txt').read_text()

static_images = Path('../../app/static/images/')


In [255]:
repo = 'html-css'
branch = 'v2.0-param-edits'

for image in all_images.findall(markdown):
    filename = image[0].split('/')[-1]
    url = f'https://raw.githubusercontent.com/DHRI-Curriculum/{repo}/{branch}/images/{filename}'
    local_file = static_images / Path(repo) / filename
    if not local_file.parent.exists(): local_file.parent.mkdir(parents=True)
    if not local_file.exists():
        r = requests.get(url)
        if r.status_code == 200:
            with open(local_file, 'wb+') as f:
                for chunk in r:
                    f.write(chunk)
        elif r.status_code == 404:
            url = f'https://raw.githubusercontent.com/DHRI-Curriculum/{repo}/{branch}/sections/images/{filename}' # secondary, search in sections directory
            r = requests.get(url)
            if r.status_code == 200:
                with open(local_file, 'wb+') as f:
                    for chunk in r:
                        f.write(chunk)
            elif r.status_code == 404:
                print(f"Warning: Could not download image {filename} (not found)")
            elif r.status_code == 403:
                print(f"Warning: Could not download image {filename} (not allowed)")
        elif r.status_code == 403:
            print(f"Warning: Could not download image {filename} (not allowed)")